# Using Data Base in gstlearn

In this preamble, we load the **gstlearn** library.

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import matplotlib.pyplot as plt
import numpy as np
import os
import urllib.request

This is the (non-exhaustive) list of classes (of objects) in gstlearn:

* Db, DbGrid: numerical data base
* DirParam, VarioParam and Vario: experimental variograms
* Model: variogram model
* Neigh: neighborhood
* Anam: gaussian anamorphosis
* Polygon: 2-D polygonal shapes
* Rule: lithotype rule for thresholds used for truncated plurigaussian models

# Importing External File

## Loading CSV file

You should download the ASCII file called *Scotland_Temperatures.csv* (organized as a CSV file) and store it on your disk in temporary directory. In this example, the file (called *temp_csv*) is provided as a *CSV* format file. We load it into a data frame (names *datcsv*) using the relevant Python command.

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.csv'
temp_csv, head = urllib.request.urlretrieve(url)

In [ ]:
import pandas as pd
datcsv = pd.read_csv(temp_csv)
datcsv

We can check the contents of the data frame (by simply typing its name) and see that it contains four columns (respectively called *Longitude*, *Latitude*, *Elevation*, *January_temp*) and 236 rows (header line excluded).

Note that the last column contains several values called *MISS*: this corresponds to the absence of information.

## Creating Db File from CSV

We now want to load this information in order to obtain a data base of the gstlearn package (or *Db*) that will be called *dat*. This operation can be performed directly by reading the CSV file again and load it directly into a Db.

Note that we introduce a *CSVformat* description where we can specifiy the specificities of the file to be read, in particular we can tell how to spell the conventional value used for coding missing information.

In [ ]:
csv = gl.CSVformat.create(flagHeader=True, naString = "MISS")
dat = gl.Db.createFromCSV(temp_csv, csv=csv)
dat

## Importing Db File

A last solution is to import it directly from the set of demonstration files (provided together with the package and called *temp_nf*) and stored in a specific format (Neutral file). 

These *NF* (or neutral file) are currently used for serialization of the gstlearn objects. They will probably be replaced in the future by a facility backuping the whole workspace in one step.

Note that the contents of the Db is slightly different from the result obtained when reading from CSV. Essentially, some variables have a *Locator* field defined, some do not. This concept will be described later in this chapter and the difference can be ignored.

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.NF'
temp_nf, head = urllib.request.urlretrieve(url)
dat = gl.Db.createFromNF(temp_nf)
dat

# Discovering Db

## The Db class

Typing the name of the object automatically launches the **display**. It is equivalent to simply typing the name of the object (at the end of a cell in a jupyter notebook).


In [ ]:
dat.display()

There, we can check that the 4 initial fields have been considered, in addition to a firs one, automatically called *rank*, for a total of 5 columns (the information regarding *UID* will not be addressed in this chapter).

We can check that each field is assigned to a numbered *Column*. Finally the total number of samples is 236 as expected.

In addition, some interesting information tells you that this data base corresponds to a 2-D dimension one: this will be described later together with the use of the *Locator* information.


To get more information on the contents of the Db, it is possible to use the *DbStringFormat* option and to use use through the *display* method. There are several ways to specify the type of information that is searched for (see the documentation of this class for details): typically here we ask for statistics but restrict them to a list of variables 

In [ ]:
dbfmt = gl.DbStringFormat.createFromFlags(flag_stats=True, names=["Elevation", "January_temp"])
dat.display(dbfmt)

We can also consider the data base as a data frame and use the *[  ]* assessors. The following usage show the whole contents of the data base.

In [ ]:
dat[:]

We can access to one or several variables. Note that the contents of the Column corresponding to the target variable (i.e. *January_temp*) is produced as a series of values (printed along a line). 

Also note the presence of samples with *NA* corresponding to those where the target variable is not informed.

In [ ]:
dat["January_temp"]

But it can be more restrictive as in the following paragraph, where we only consider the samples 10 to 15, and only consider the variables *Latitude* and *Elevation*.

In [ ]:
dat[10:15, ["Latitude", "Elevation"]]

We can also replace the variable *Name* by their *Column* rank. Although this is not recommanded as the Column number may vary over time.

In [ ]:
dat[10:15, 2:4]

Please also note the feature that a variable whose name does not exist (*newvar*) in the data base, is created on the fly. Also note that variables may be specified with names referred to using traditional regexp expressions (i.e. the symbol '*' replaces any list of characters):

In [ ]:
dat["newvar"] = 12.3 * dat["Elevation"] - 2.1 * dat["*temp"]
dat

## Locators

The locators are used to specify the **role** assigned to a Column for the rest of the study (unless changed further). The locator is characterized by its name (*Z* for a variable and *X* for a coordinate) within the Enumeration *ELoc* and its rank.

In [ ]:
dat.setLocators(["Longitude","Latitude"], gl.ELoc.X)
dat.setLocator("*temp", gl.ELoc.Z, cleanSameLocator=True)
dat

As can be seen in the printout, variables *Latitude* and *Longitude* have been designated as coordinates (pay attention to the order) and *January_temp* is the (unique) variable. Therefore any subsequent step will be performed as a monovariate 2-D process.

The locator is translated into a *letter*,*number* pair for better legibility: e.g. *x1* for the first coordinate.


## Plotting a Db

Plot the contents of a Db using functions of the **plot.py** package. The proportional option is used to represent to **january_temp** variable.

In [ ]:
fig, ax = gp.initGeographic()
ax.symbol(dat, name_size="*temp", flagLegend=True, legendName="Temperature")
ax.decoration(title="My Data Base", xlabel="Easting", ylabel="Northing")
plt.show()

A more elaborated graphic representation displays the samples with a symbol proportional to the Elevation and a color representing the Temperature.

In [ ]:
fig, ax = gp.initGeographic()
ax.symbol(dat, name_size="Elevation", name_color="*temp", flagLegend=True, legendName="Elevation")
ax.decoration(title="My Data Base", xlabel="Easting", ylabel="Northing")
plt.show()

# Grid Data Base

On the same area, a terrain model is available (as a demonstration file available in the package distribution). We first download it as an element of a data base defined on a grid support (*DbGrid*).

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Elevations.NF'
elev_nf, head = urllib.request.urlretrieve(url)
grid = gl.DbGrid.createFromNF(elev_nf)
grid

We can check that the grid is constituted of 81 columns and 137 rows, or 11097 grid cells.


## Selection

We can check the presence of a variable (called *inshore*) which is assigned to the *sel* locator: this corresponds to a *Selection* which acts as a binary filter: some grid cells are active and others are masked off. The count of active samples is given in the previous printout (3092). This selection remains active until it is replaced or deleted (there may not be more than one selection defined at a time per data base). This is what can be seen in the following display where we represent the *Elevation* only within the *inshore* selection.

Note that any variable can be considered as a Selection: it must simply be assigned to the *sel* locator using the *setLocator* variable described earlier.

In [ ]:
fig, ax = gp.initGeographic()
ax.raster(grid, name="Elevation")
ax.decoration(title="My Grid", xlabel="Easting", ylabel="Northing")
plt.show()

On this final plot, we combine grid and point representations.

In [ ]:
fig, ax = gp.initGeographic()
ax.raster(grid, name="Elevation")
ax.symbol(dat, name_size="*temp", flagLegend=True, legendName="Temperature", c="yellow")
ax.decoration(title="My Grid", xlabel="Easting", ylabel="Northing")
plt.show()